In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC ## Demo_inference_Notebook1
# MAGIC This notebook performs inference using a previously trained and logged model from MLflow.
# MAGIC 
# MAGIC - Uses mock input
# MAGIC - Loads model from MLflow
# MAGIC - Logs inference metadata
# MAGIC - Follows staff-level modular MLOps best practices

# Mock dbutils if not in Databricks environment
try:
    if not dbutils:
        raise NameError
except NameError:
    class DBUtilsMock:
        def notebook(self):
            return self

        def getContext(self):
            return self

        def userName(self):
            return self

        def get(self):
            return "mock_user@example.com"

    dbutils = DBUtilsMock()

# Now proceed with the rest of the notebook


In [0]:
# COMMAND ----------
%pip install pandas scikit-learn mlflow --quiet


In [0]:
# COMMAND ----------
# Section 2 - Imports & Setup

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from datetime import datetime
import json
import os
from urllib.parse import urlparse

# Load config
CONFIG_PATH = "mlops_config.json"
if os.path.exists(CONFIG_PATH):
    with open(CONFIG_PATH, "r") as f:
        config = json.load(f)
        repo_name = config.get("repo_name", "unknown-repo")
        branch_name = config.get("branch_name", "unknown")
else:
    raise FileNotFoundError(f"❌ Config file not found: {CONFIG_PATH}")

# Derive env from branch
env = "prod" if branch_name == "main" else "dev"

# Get current user
user_email = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()


In [0]:
# COMMAND ----------
# Section 3 - Load Model

# Build experiment path
experiment_path = f"/Users/{user_email}/{repo_name}_train_{env}"

# Get latest run with a logged model
client = MlflowClient()
experiment = client.get_experiment_by_name(experiment_path)
runs = client.search_runs(experiment.experiment_id, order_by=["start_time DESC"], max_results=1)

if not runs:
    raise ValueError("❌ No MLflow runs found with logged models.")

latest_run = runs[0]
model_uri = f"runs:/{latest_run.info.run_id}/model"
model = mlflow.sklearn.load_model(model_uri)
print(f"✅ Loaded model from: {model_uri}")


In [0]:
# COMMAND ----------
# Section 4 - Generate Mock Input

# Simulate input data (2 features)
input_data = pd.DataFrame({
    "feature1": [0.15, 0.60],
    "feature2": [0.75, 0.30]
})


In [0]:
# COMMAND ----------
# Section 5 - Inference

predictions = model.predict(input_data)
print("Predictions:", predictions)


In [0]:
# COMMAND ----------
# Section 5.1 - Save predictions to Unity Catalog table (dynamically)

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# Step 1: Prepare prediction DataFrame
pred_df = pd.DataFrame({"prediction": predictions})
spark_pred_df = spark.createDataFrame(pred_df)

# Step 2: Dynamically find a UC catalog + schema
catalogs = [row['catalog'] for row in spark.sql("SHOW CATALOGS").collect()]
selected_catalog = None
selected_schema = None

for cat in catalogs:
    try:
        schemas = spark.sql(f"SHOW SCHEMAS IN {cat}").collect()
        for s in schemas:
            schema_name = s['databaseName']
            if repo_name in schema_name or schema_name.lower() == "default":
                selected_catalog = cat
                selected_schema = schema_name
                break
        if selected_catalog: break
    except Exception:
        continue  # skip non-accessible/system catalogs

# Step 3: Build full UC table path
if not selected_catalog or not selected_schema:
    raise Exception("❌ Could not find a valid UC catalog/schema.")

uc_output_table = f"{selected_catalog}.{selected_schema}.inference_output"

# Step 4: Save predictions as Delta table in UC
spark_pred_df.write.mode("overwrite").format("delta").saveAsTable(uc_output_table)

print(f"✅ Inference predictions saved to Unity Catalog table: {uc_output_table}")


In [0]:
# COMMAND ----------
# Section 6 - MLflow Tracking (Inference)

inference_exp_path = f"/Users/{user_email}/{repo_name}_inference_{env}"

# Ensure experiment exists
if not client.get_experiment_by_name(inference_exp_path):
    client.create_experiment(inference_exp_path)

mlflow.set_experiment(inference_exp_path)

if mlflow.active_run():
    mlflow.end_run()

run_name = f"{repo_name}-inference-{env}"

with mlflow.start_run(run_name=run_name):
    mlflow.set_tags({
        "project": repo_name,
        "notebook": "Demo_inference_Notebook1",
        "branch": branch_name,
        "env": env,
        "owner": user_email,
        "run_type": "inference",
        "source_model_uri": model_uri,
        "date": datetime.today().strftime('%Y-%m-%d')
    })

    mlflow.log_param("input_rows", len(input_data))
    mlflow.log_param("model_uri", model_uri)

    # Optional: log predictions as artifact or sample
    prediction_sample = model.predict(input_data)
    prediction_df = pd.DataFrame(prediction_sample)
    temp_path = "/tmp/inference_output.csv"
    prediction_df.to_csv(temp_path, index=False)
    mlflow.log_artifact(temp_path)


    print(f"✅ Inference logged to MLflow run '{run_name}'")


In [0]:
# COMMAND ----------
# Section 10 - Clean Exit

print("✅ Inference completed successfully.")
